Pre-trained model download link : [15epoch](https://drive.google.com/file/d/1WNVQy4bvtg4VmDQKLF3-eJfKROltM9TI/view?usp=sharing), [25epoch](https://drive.google.com/file/d/1TVxYdy_XvyzdT3YsjT-gYar123nCU0Hj/view?usp=sharing) model

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 46kB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 3.1MB 48.5MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [9]:
#!/usr/bin/env python

import argparse
import io
import os
import sys

import tensorflow as tf

#SCRIPT_PATH = os.path.dirname(os.path.abspath(__file__))

# Default paths.
'''DEFAULT_LABEL_FILE = os.path.join(
    SCRIPT_PATH, '/gdrive/My Drive/CS470/teamai/labels/2350-common-hangul.txt'
)
DEFAULT_GRAPH_FILE = os.path.join(
    SCRIPT_PATH, '/gdrive/My Drive/CS470/teamai/saved-model/optimized_hangul_tensorflow.pb'
)'''


def read_image(file):
    """Read an image file and convert it into a 1-D floating point array."""
    file_content = tf.read_file(file)
    image = tf.image.decode_jpeg(file_content, channels=1)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.reshape(image, (1, 64*64))
    return image


def classify(img_file, label_file, graph_file):
    """Classify a character.

    This method will import the saved model from the given graph file, and will
    pass in the given image pixels as input for the classification. The top
    five predictions will be printed.
    """
    labels = io.open(label_file,
                     'r', encoding='utf-8').read().splitlines()

    if not os.path.isfile(img_file):
        print('Error: Image %s not found.' % img_file)
        sys.exit(1)

    # Load graph and parse file.
    with tf.gfile.GFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name='hangul-model',
            producer_op_list=None
        )

    # Get relevant nodes.
    x = graph.get_tensor_by_name('hangul-model/input:0')
    y = graph.get_tensor_by_name('hangul-model/output:0')
    keep_prob = graph.get_tensor_by_name('hangul-model/keep_prob:0')

    image = read_image(img_file)
    sess = tf.InteractiveSession()
    image_array = sess.run(image)
    sess.close()
    with tf.Session(graph=graph) as graph_sess:
        predictions = graph_sess.run(y, feed_dict={x: image_array,
                                                   keep_prob: 1.0})
        prediction = predictions[0]

    # Get the indices that would sort the array, then only get the indices that
    # correspond to the top 5 predictions.
    sorted_indices = prediction.argsort()[::-1][:5]
    for index in sorted_indices:
        label = labels[index]
        confidence = prediction[index]
        return label

#change the path names in classify for your own file
# second arguement should be the label file, third arguement should be the pre-trained model
def tag(image_file):
    return classify(image_file, '/gdrive/My Drive/CS470/teamai/labels/2350-common-hangul.txt', '/gdrive/My Drive/CS470/teamai/save_model_trained/optimized_hangul_tensorflow_2350_15.pb')


In [4]:
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import glob
import pandas as pd

Input image files should be named as the correct Korean character to evaluated the accuracy of the model

In [5]:
#the path in glob should be the path to the image files, also change png into the img files state
images = glob.glob('/gdrive/MyDrive/CS470/teamai/test_letter/all_test_ex/*.jpg')

The unicode type of the parsed image path name is not the same as the model generated result. So the accuracy is now calculated by human labor.

In [10]:
#this will save the images with the tagged unicode into the directory you insert
count = 0
for img in images:
  image = img
  cvimg = cv2.imread(image)
  dst = cv2.bitwise_not(cvimg)
  cv2.imwrite( "checking.jpg", dst )
  newimg = Image.open("checking.jpg")
  resized = newimg.resize((64,64))

  data = np.array(resized)
  red, green, blue = data[:,:,0], data[:,:,1],data[:,:,2]
  for i in range(64):
    for j in range(64):
      imsi = data[i][j]
      if (int(imsi[0])+int(imsi[1])+int(imsi[2])) < 300:
        data[i][j] = [0,0,0]
  newimg1 = Image.fromarray(data)

  newimg1.save('new1.jpeg')
  tagged_letter = tag('/content/new1.jpeg')
  #the path will be the directory you want to save the img files
  name = img.split('/')[-1].split('.')[0]
  print(tagged_letter, name)
  count += 1
print(count)
  

서 세
기 기
유 류
의 의
과 과
둣 술
읜 은
인 인
생 생
학 학
큰 을
임 임
다 다
아 아
긴 고
도 도
보 보
부 부
책 책
꼬 언
네 니
김 그
끈 러
나 나
최 최
귄 근
눈 눈
닌 신
곡 속
발 발
전 전
괴 리
핸 행
까 까
지 지
의 미
유 우
하 하
누 수
울 물
져 적
해 해
꾜 좌
되 되
겆 것
에 에
즉 즉
절 질
풍 풍
쟈 주
영 영
챦 향
여 예
어 어
들 들
묘 요
자 자
판 만
외 외
나 내
난 난
게 게
갑 람
롭 롭
사 사
화 화
목 목
한 한
가 가
정 정
더 더
인 일
셔 서
종 중
네 비
면 면
윤 윤
덕 덕
히 히
롯 롯
성 성
얻 엄
죤 존
특 특
간 간
관 관
명 명
닐 실
앗 앗
채 채
녀 려
닐 실1
챦 않
졔 제
치 치
뿐 뿐
쁠 불
개 개
귀 귀
뚜 뛰
등 등
판 단
능 능
놓 놓
례 례
오 모
읖 른
여 여
야 야
있 있
쁨 쁨
친 친
켜 켜
총 충
첫 청
회 회
현 현
할 할
표 또
루 루
여 며
못 못
본 본
됴 순
십 심
쟝 장
준 준
죡 족
접 집
키 키
꾐 필
전 진
들 를
보 복
화 후
135
